# Lab6 决策树分类器
- 姓名：彭钰钊
- 学号：2110756
- 专业：计算机科学与技术
- GitHub链接：https://github.com/Yuzhao-P/Machine-Learning
    > 上传历次实验数据文件和.ipynb文件，同时在README.md中更新有关实验的相关知识整理。
## 实验要求

### 基本要求
a.	基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；  
b.	基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
### 中级要求
a.  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；  
b.	对测试集Watermelon-test2进行预测，输出分类精度；
### 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

***截止日期：12月15日***

- 以学号+姓名(6)的命名形式打包实验代码+实验报告发送到邮箱18329300691@163.com
- 推荐使用.ipynb文件格式

### 导入所需要的包

In [1]:
import numpy as np 
import pandas as pd
import copy

### 导入数据

In [2]:
train1 = pd.read_csv("Watermelon-train1.csv", encoding="ansi")
train2 = pd.read_csv("Watermelon-train2.csv", encoding="ansi")
test1 = pd.read_csv("Watermelon-test1.csv", encoding="ansi")
test2 = pd.read_csv("Watermelon-test2.csv", encoding="ansi")

In [3]:
train1

,编号,色泽,根蒂,敲声,纹理,好瓜
0,1,青绿,蜷缩,浊响,清晰,是
1,2,乌黑,蜷缩,沉闷,清晰,是
2,3,乌黑,蜷缩,浊响,清晰,是
3,4,青绿,蜷缩,沉闷,清晰,是
4,5,浅白,蜷缩,浊响,清晰,是
5,6,青绿,稍蜷,浊响,清晰,是
6,7,乌黑,稍蜷,浊响,稍糊,是
7,8,乌黑,稍蜷,浊响,清晰,是
8,9,乌黑,稍蜷,沉闷,稍糊,否
9,10,青绿,硬挺,清脆,清晰,否


In [4]:
train2

,编号,色泽,根蒂,敲声,纹理,密度,好瓜
0,1,青绿,蜷缩,浊响,清晰,0.697,是
1,2,乌黑,蜷缩,沉闷,清晰,0.774,是
2,3,乌黑,蜷缩,浊响,清晰,0.634,是
3,4,青绿,蜷缩,沉闷,清晰,0.608,是
4,5,浅白,蜷缩,浊响,清晰,0.556,是
5,6,青绿,稍蜷,浊响,清晰,0.403,是
6,7,乌黑,稍蜷,浊响,稍糊,0.481,是
7,8,乌黑,稍蜷,浊响,清晰,0.437,是
8,9,乌黑,稍蜷,沉闷,稍糊,0.666,否
9,10,青绿,硬挺,清脆,清晰,0.243,否


In [5]:
test1

,编号,色泽,根蒂,敲声,纹理,好瓜
0,1,浅白,蜷缩,浊响,清晰,是
1,2,乌黑,稍蜷,沉闷,清晰,是
2,3,乌黑,蜷缩,沉闷,清晰,是
3,4,青绿,蜷缩,沉闷,稍糊,是
4,5,浅白,蜷缩,浊响,清晰,是
5,6,青绿,稍蜷,浊响,清晰,否
6,7,乌黑,稍蜷,浊响,稍糊,否
7,8,青绿,稍蜷,浊响,模糊,否
8,9,乌黑,稍蜷,沉闷,稍糊,否
9,10,青绿,硬挺,清脆,模糊,否


In [6]:
test2

,编号,色泽,根蒂,敲声,纹理,密度,好瓜
0,1,乌黑,稍蜷,浊响,清晰,0.403,是
1,2,青绿,稍蜷,浊响,稍糊,0.481,是
2,3,乌黑,稍蜷,浊响,清晰,0.337,是
3,4,乌黑,稍蜷,沉闷,稍糊,0.666,否
4,5,青绿,硬挺,清脆,清晰,0.243,否


In [7]:
# 构建基本节点类
class Node:
    def __init__(self, data=None, labels=None, threshold=None):
        self.data = data
        self.labels = labels
        self.threshold = threshold  # 针对连续性属性的阈值
        self.children = {}

    def is_leaf(self):
        return self.labels is not None

## 基本要求
我们的设计思路如下：
1. 编写信息熵计算函数cal_entropy
2. 编写信息增益计算函数cal_infor_gain
3. 编写ID3算法主体构建决策树DT_ID3
4. 在测试集上预测并输出精度

### 信息熵
**熵（entropy）概念：**  
1948年，香农提出了“信息熵”的概念。在信息论与概率统计中，熵是表示随机变量不确定性的量。X是⼀个取值为有限个的离散随机变量，
$$ H(X)=-\sum_{i=1}^{n} p\left(x_{i}\right) \log p\left(x_{i}\right)$$ 
$𝐻(𝑋)$就被称作随机变量𝑋的熵，它表示随机变量不确定的度量。熵取值越大，随机变量不确定性越大。当随机变量为均匀分布时，熵最大。当某一状态概率取值为1时，熵的值为零。

In [8]:
# 计算信息熵
def cal_entropy(labels):
    """
        计算目标值的信息熵。

        参数:
        - labels: 数组，形状为 (n_samples,)
            目标值。

        返回:
        - 浮点数
        目标值的信息熵。
    """
    classes, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    
    return entropy_value

### 信息增益
- 条件熵 $𝐻(𝑌∣𝑋)$ ：  
	表示在已知随机变量𝑋的条件下随机变量𝑌的不确定性，定义为给定𝑋条件下𝑌的条件概率分布的熵对𝑋的数学期望:
$$H(Y \mid X)=\sum_{x} p(x) H(Y \mid X=x) =-\sum_{x} p(x) \sum_{y} p(y \mid x) \log p(y \mid x)$$

- 特征𝐴对数据集𝐷的信息增益就是熵$𝐻(𝐷)$与条件熵$𝐻(𝐷|𝐴)$之差:
$$infor\_gain(𝐷∣𝐴) = 𝐻(𝐷)−𝐻(𝐷∣𝐴)$$

表示已知特征𝐴的信息而使得数据集𝐷的信息不确定减少的程度。信息增益越大的特征代表其具有更强的分类能力，所以我们就要**选择能够使数据的不确定程度减少最多的特征**，也就是信息增益最大的特征。

In [9]:
# 计算信息增益
def cal_infor_gain(data, labels, feature_index):
  """
    计算特定特征的信息增益。

    参数:
    - data: 二维数组，形状为 (n_samples, n_features)
      输入特征数据。
    - labels: 数组，形状为 (n_samples,)
      目标值。
    - feature_index: 整数
      要计算信息增益的特征的索引。

    返回:
    - 浮点数
      指定特征的信息增益。
  """
  unique_values = np.unique(data[feature_index])
  weighted_entropy = 0

  for value in unique_values:
    # 获取索引
    subset_indices = data[feature_index] == value
    # 获取子集
    subset_labels = labels[subset_indices]
    # 加权熵计算
    weighted_entropy += len(data[subset_indices]) / len(data) * cal_entropy(subset_labels)

  return cal_entropy(labels) - weighted_entropy

### ID3算法构建决策树
基本思路：
1. 判断基准条件： 如果所有样本属于同一类别，创建一个叶子节点，并将该类别作为目标值返回。
2. 递归停止条件： 如果没有可用的特征，创建一个叶子节点，并将目标值中的众数作为目标值返回。
3. 选择最佳特征： 根据信息增益选择最佳特征，以此作为当前节点的判断依据。
4. 递归构建子树： 对于最佳特征的每个取值，递归调用 DT_ID3 函数，构建子树。
5. 返回节点： 返回当前节点，其中包含对应于最佳特征取值的子树。

In [10]:
def DT_ID3(data, labels, features):
    """
        使用ID3算法构建决策树。
        
        参数:
        - data
            数据特征
        - labels
            数据标签
        - features
            特征名称

        返回:
        - Node
            决策树的根节点。
    """
    # 创建节点
    node = Node()

    # 如果所有样本属于同一类别，创建叶子节点
    if len(np.unique(labels)) == 1:
        node.labels = labels.iloc[0]
        return node
        
    # 如果没有特征可用，创建叶子节点，其中包含目标值的众数
    if len(features) == 0:
        node.labels = labels.mode().iloc[0]
        return node

    # 选择最佳划分特征
    gains = [cal_infor_gain(data, labels, feature) for feature in features]
    best_feature = features[np.argmax(gains)]
    node.data = best_feature
    # 递归构建子树
    unique_values = np.unique(data[best_feature])
    for value in unique_values:
        subset_indices = data[best_feature] == value
        child_node = DT_ID3(data[subset_indices].drop(columns=[best_feature]), labels[subset_indices], features.drop(best_feature))
        node.children[value] = child_node

    return node

In [11]:
def print_decision_tree(node, depth=0, feature_name='Root'):
    """
        递归打印决策树的结构。

        参数:
        - node: Node
            当前节点。
        - depth: int
            当前节点的深度，用于缩进输出。
        - feature_name: str
            当前节点的特征名称。

        返回:
        - None
    """

    # 缩进，根据深度增加空格
    indentation = "  " * depth

    # 打印当前节点信息
    if node.data is not None:
        print(indentation + f"Feature: {node.data}")
    else:
        print(indentation + f"Label: {node.labels}")
    # 递归打印子节点
    for value, child_node in node.children.items():
        print(indentation + f"  Value = {value}:")
        print_decision_tree(child_node, depth + 1, feature_name)

In [12]:
def predict_single_sample(node, sample):
    """
        递归预测单个样本。

        参数:
        - node: Node
            当前节点。
        - sample: dict
            包含特征的样本。

        返回:
        - label
            预测的类别。
    """

    # 如果是叶子节点，返回类别
    if node.labels is not None:
        return node.labels

    # 获取当前特征的取值
    value = sample.get(node.data)

    # 处理连续性属性
    if node.threshold is not None:
        if value <= node.threshold:
            return predict_single_sample(node.children['<='], sample)
        else:
            return predict_single_sample(node.children['>'], sample)
    else:
        # 处理离散属性
        if value in node.children:
            return predict_single_sample(node.children[value], sample)
        else:
        # 如果取值未在训练数据中出现，可以根据情况处理，例如返回默认值或抛出异常
            return None
def predict_batch_samples(tree, data):
    """
        预测批量样本。

        参数:
        - tree: Node
            决策树的根节点。
        - data: pd.DataFrame
            包含特征的数据框。

        返回:
        - predictions: list
            预测的类别列表。
    """

    predictions = []
    for index, sample in data.iterrows():
        prediction = predict_single_sample(tree, sample)
        predictions.append(prediction)
    return predictions

In [13]:
# 构建ID3决策树
data = train1.iloc[:, 1:5]
labels = train1.iloc[:, -1]
features = train1.columns[1:5]
root = DT_ID3(data, labels, features)
# 测试集预测
print("=========================ID3-决策树=========================")
print_decision_tree(root)
prediction = predict_batch_samples(root, test1)
print(f"预测结果: {prediction}")
print(f"实际结果: {test1.iloc[:, -1].tolist()}")
accuracy0 = sum(prediction == test1.iloc[:, -1]) / len(test1)
print(f"ID3-决策树在Watermelon-test1.csv上的分类准确率: {accuracy0 * 100:.2f}%")

=========================ID3-决策树=========================
Feature: 纹理
  Value = 模糊:
  Label: 否
  Value = 清晰:
  Feature: 根蒂
    Value = 硬挺:
    Label: 否
    Value = 稍蜷:
    Label: 是
    Value = 蜷缩:
    Label: 是
  Value = 稍糊:
  Feature: 色泽
    Value = 乌黑:
    Feature: 敲声
      Value = 沉闷:
      Label: 否
      Value = 浊响:
      Label: 是
    Value = 浅白:
    Label: 否
    Value = 青绿:
    Label: 否
预测结果: ['是', '是', '是', '否', '是', '是', '是', '否', '否', '否']
实际结果: ['是', '是', '是', '是', '是', '否', '否', '否', '否', '否']
ID3-决策树在Watermelon-test1.csv上的分类准确率: 70.00%


## 中级要求
信息增益偏向于选择取值较多的特征进行划分。⽐如学号这个特征，每个学生都有一个不同的学号，如果根据学号对样本进行分类，则每个学生都属于不同的类别，这样是没有意义的。而C4.5在生成过程中，用**信息增益比**来选择特征，可以校正这个问题。

**信息增益比**公式：$$gain\_ratio(𝐷∣𝐴) = \frac{infor\_gain(𝐷∣𝐴)}{IV(A)}\\IV(A) = -\sum\limits^K_{k=1}\frac{|D_k|}{|D|}\times\log _2\frac{|D_k|}{|D|}$$

In [14]:
# 信息增益比计算
def cal_infor_gain_ratio(X, y, feature, threshold=None):
    if threshold is not None:
        # 处理连续性属性
        subset_indices = X[feature] <= threshold
        subset1 = y[subset_indices]
        subset2 = y[~subset_indices]
        weight1 = len(subset1) / len(X)
        weight2 = len(subset2) / len(X)
        subset_entropy = weight1 * cal_entropy(subset1) + weight2 * cal_entropy(subset2)
    else:
        # 处理离散属性
        unique_values = np.unique(X[feature])
        subset_entropy = 0
        for value in unique_values:
            subset_indices = X[feature] == value
            subset = y[subset_indices]
            weight = len(subset) / len(X)
            subset_entropy += weight * cal_entropy(subset)

    entropy_before = cal_entropy(y)
    gain = entropy_before - subset_entropy
    # print("gain:",gain)
    # 计算分裂信息
    # split_info = -np.sum((len(X[X[feature] == value]) / len(X)) * np.log2(len(X[X[feature] == value]) / len(X))
    #                     for value in np.unique(X[feature]))
    if threshold is not None:
        split_info = -(len(X[X[feature] <= threshold]) / len(X)) * np.log2(len(X[X[feature] <= threshold]) / len(X)) - (len(X[X[feature] > threshold]) / len(X)) * np.log2(len(X[X[feature] > threshold]) / len(X))
    else:
        split_info = -sum((len(X[X[feature] == value]) / len(X)) * np.log2(len(X[X[feature] == value]) / len(X)) for value in np.unique(X[feature]))

    # 计算信息增益比
    gain_ratio = gain / split_info if split_info != 0 else 0

    return gain_ratio

In [15]:
def DT_C45(X, y, features):
    # 创建节点
    node = Node()

    # 如果所有样本属于同一类别，则将节点标记为该类别
    if len(np.unique(y)) == 1:
        node.labels = y.iloc[0]
        return node

    # 如果没有更多特征可供划分，则将节点标记为样本中最常见的类别
    if len(features) == 0:
        node.labels = y.mode().iloc[0]
        return node

    best_feature = None
    best_threshold = None
    best_gain_ratio = -1

    # 遍历每个特征，选择最佳的划分属性和阈值
    for feature in features:
        if X[feature].dtype == np.float64:
            # 处理连续性属性
            unique_values = np.unique(X[feature])
            # print(unique_values)
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2  # 中值作为阈值
            # print(thresholds)
            for threshold in thresholds:
                gain_ratio = cal_infor_gain_ratio(X, y, feature, threshold)
                # print(feature, gain_ratio)
                if gain_ratio > best_gain_ratio:
                    best_gain_ratio = gain_ratio
                    best_feature = feature
                    best_threshold = threshold
                    # print(best_threshold)
        else:
            # 处理离散属性
            gain_ratio = cal_infor_gain_ratio(X, y, feature)
            # print(feature, gain_ratio)
            if gain_ratio > best_gain_ratio:
                best_gain_ratio = gain_ratio
                best_feature = feature

    # print(best_feature, best_gain_ratio, best_threshold)

    node.data = best_feature
    node.threshold = best_threshold

    if best_feature is not None:
        # 递归构建子树
        if best_threshold is not None:
            subset_indices = X[best_feature] <= best_threshold
            # child_node1 = DT_C45(X[subset_indices], y[subset_indices], features.drop(best_feature))
            # child_node2 = DT_C45(X[~subset_indices], y[~subset_indices], features.drop(best_feature))
            child_node1 = DT_C45(X[subset_indices], y[subset_indices], features)
            child_node2 = DT_C45(X[~subset_indices], y[~subset_indices], features)
            node.children['<='] = child_node1
            node.children['>'] = child_node2
        else:
            unique_values = np.unique(X[best_feature])
            for value in unique_values:
                subset_indices = X[best_feature] == value
                child_node = DT_C45(X[subset_indices], y[subset_indices], features.drop(best_feature))
                node.children[value] = child_node

    return node

In [16]:
def print_c45_tree(node, depth=0, value=None):
    indentation = "  " * depth
    if node.data is not None:
        if node.threshold is not None:
            print(indentation + f"Feature: {node.data} <= {node.threshold}")
        else:
            print(indentation + f"Feature: {node.data}")
    else:
        print(indentation + f"Label: {node.labels}")
    if value is not None:
        print(indentation + f"  Value: {value}")
    for child_value, child_node in node.children.items():
        print_c45_tree(child_node, depth + 1, child_value)

In [17]:
# 构建C4.5决策树
X = train2.iloc[:, 1:6]
y = train2.iloc[:, -1]
features_ = train2.columns[1:6]
root_ = DT_C45(X, y, features_)
# 测试集预测
print("=========================C45-决策树=========================")
print_c45_tree(root_)
prediction_ = predict_batch_samples(root_, test2)
print(f"预测结果: {prediction_}")
print(f"实际结果: {test2.iloc[:, -1].tolist()}")
accuracy_ = sum(prediction_ == test2.iloc[:, -1]) / len(test2)
print(f"C45-决策树在Watermelon-test2.csv上的分类准确率: {accuracy_ * 100:.2f}%")

=========================C45-决策树=========================
Feature: 密度 <= 0.3815
  Label: 否
    Value: <=
  Feature: 纹理
    Value: >
    Label: 否
      Value: 模糊
    Label: 是
      Value: 清晰
    Feature: 密度 <= 0.56
      Value: 稍糊
      Label: 是
        Value: <=
      Label: 否
        Value: >
预测结果: ['是', '是', '否', '否', '否']
实际结果: ['是', '是', '是', '否', '否']
C45-决策树在Watermelon-test2.csv上的分类准确率: 80.00%


## 高级要求
决策树剪枝是一种优化决策树模型的方法，目的是提高其泛化能力，避免过拟合。在训练决策树时，为了最大程度地拟合训练数据，可能会生成一个过于复杂的树，这可能导致在新数据上表现不佳。剪枝通过去除一些节点或子树来简化树，从而改善模型的性能。

以下是一些常见的决策树剪枝方法：

### 预剪枝（Pre-pruning）：

在树的构建过程中，在每个节点处做出剪枝决策。通常有以下一些预剪枝条件：

1. 限制树的最大深度。
2. 限制每个叶子节点的最小样本数。
3. 限制每个节点的最小样本数。
### 后剪枝（Post-pruning）：
允许树过度拟合训练数据，然后再去除不必要的部分。通常采用以下一些后剪枝方法：

1. 验证集剪枝：将数据集分为训练集和验证集，构建完整的决策树，然后通过验证集的性能来判断是否剪枝。
2. 成本复杂度剪枝（Cost-complexity pruning）：引入一个复杂度参数，通过最小化损失函数和复杂度之和来选择剪枝点。这个方法可以使用交叉验证来确定复杂度参数的值。

我们选择后剪枝——验证集剪枝，使用验证集性能判断是否剪枝。

In [18]:
def accuracy(node, test):
    predictions = predict_batch_samples(node, test)
    acc = sum(predictions == test.iloc[:, -1]) / len(test)
    return acc

In [19]:
# 递归后剪枝
def prune_tree(node, validation_data):
    if node.is_leaf():
        return node

    accuracy_before_prune = accuracy(node, validation_data)
    # 递归剪枝子树
    for child_key in node.children:
        prune_tree(node.children[child_key], validation_data)

    # 保存未剪枝前的节点状态
    original_data = node.data
    original_labels = node.labels
    original_threshold = node.threshold
    original_children = node.children.copy()

    # 剪枝：将当前节点变为叶子节点
    node.data = None
    node.labels = majority_class(validation_data)
    node.threshold = None
    node.children = {}

    # 计算剪枝后的性能
    accuracy_after_prune = accuracy(node, validation_data)

    # 比较剪枝前后的性能，决定是否还原剪枝前的状态
    if accuracy_after_prune >= accuracy_before_prune:
        # 保留剪枝后的状态
        return node
    else:
        # 还原剪枝前的状态
        node.data = original_data
        node.labels = original_labels
        node.threshold = original_threshold
        node.children = original_children
        return node

# 计算节点类别中样本数最多的类别
def majority_class(data):
    return data.iloc[:, -1].mode()[0]

In [20]:
r = copy.deepcopy(root)
r = prune_tree(r, test1)
print("=========================ID3-决策树=========================")
print_decision_tree(r)
prediction111 = predict_batch_samples(r, test1)
print(f"预测结果: {prediction111}")
print(f"实际结果: {test1.iloc[:, -1].tolist()}")
accuracy111 = sum(prediction111 == test1.iloc[:, -1]) / len(test1)
print(f"ID3-决策树【剪枝】在Watermelon-test1.csv上的分类准确率: {accuracy111 * 100:.2f}%")
print(f"ID3-决策树【未剪枝】在Watermelon-test1.csv上的分类准确率: {accuracy0 * 100:.2f}%")

=========================ID3-决策树=========================
Feature: 纹理
  Value = 模糊:
  Label: 否
  Value = 清晰:
  Feature: 根蒂
    Value = 硬挺:
    Label: 否
    Value = 稍蜷:
    Label: 是
    Value = 蜷缩:
    Label: 是
  Value = 稍糊:
  Label: 否
预测结果: ['是', '是', '是', '否', '是', '是', '否', '否', '否', '否']
实际结果: ['是', '是', '是', '是', '是', '否', '否', '否', '否', '否']
ID3-决策树【剪枝】在Watermelon-test1.csv上的分类准确率: 80.00%
ID3-决策树【未剪枝】在Watermelon-test1.csv上的分类准确率: 70.00%


In [21]:
r_ = copy.deepcopy(root_)
r_ = prune_tree(r_, test2)
print("=========================C45-决策树=========================")
print_decision_tree(r_)
prediction222 = predict_batch_samples(r_, test2)
print(f"预测结果: {prediction222}")
print(f"实际结果: {test2.iloc[:, -1].tolist()}")
accuracy222 = sum(prediction222 == test2.iloc[:, -1]) / len(test2)
print(f"C45-决策树【剪枝】在Watermelon-test2.csv上的分类准确率: {accuracy222 * 100:.2f}%")
print(f"C45-决策树【未剪枝】在Watermelon-test2.csv上的分类准确率: {accuracy_ * 100:.2f}%")

=========================C45-决策树=========================
Feature: 密度
  Value = <=:
  Label: 否
  Value = >:
  Feature: 纹理
    Value = 模糊:
    Label: 否
    Value = 清晰:
    Label: 是
    Value = 稍糊:
    Feature: 密度
      Value = <=:
      Label: 是
      Value = >:
      Label: 否
预测结果: ['是', '是', '否', '否', '否']
实际结果: ['是', '是', '是', '否', '否']
C45-决策树【剪枝】在Watermelon-test2.csv上的分类准确率: 80.00%
C45-决策树【未剪枝】在Watermelon-test2.csv上的分类准确率: 80.00%


### 分析
**ID3决策树(剪枝前)**
```
                    (纹理)   
    +-----------------+-----------------+
[模糊]              [清晰]             [稍糊]
    +                 +                 +
  (否)              (根蒂)             (色泽)
               +------+------+    +-----+-----+
            [硬挺] [稍蜷] [蜷缩][乌黑] [浅白] [青绿]
               +      +      +    +     +     +
             (否)    (是)   (是)(敲声)  (否)  (否)
                            +-----+-----+
                        [沉闷]         [浊响]
                            +           +
                           (否)        (是)
```
**ID3决策树(剪枝后)**
```
                    (纹理)   
    +-----------------+-----------------+
[模糊]              [清晰]             [稍糊]
    +                 +                 +
  (否)              (根蒂)             (否)
               +------+------+
            [硬挺] [稍蜷] [蜷缩]
               +      +      +
             (否)    (是)   (是)
```
- 如上图所示,我们可以看见在对ID3决策树进行后剪枝后,去除一些节点或子树简化了树形,降低了过拟合的误差,并且根据前面的结果输出可以看出其分类精度有所提高。
- 在对C4.5决策树进行后剪枝后,我们的树形并未发生变化,这说明我们原本的构造已经具有较高的分类精度.
- 可以发现，后剪枝（Post-pruning）相对于预剪枝（Pre-pruning）的主要优势在于：

  1. **全局优化：** 后剪枝在整个树构建完成后进行，因此可以全局优化整个树的结构。它可以更充分地利用整个数据集的信息，根据验证集或测试集的性能来调整树的结构。
  2. **适应性更强：** 后剪枝允许树在训练集上过拟合，然后再通过剪枝来降低模型的复杂度，提高泛化能力。这种方法更适应不同数据分布和特征的变化，因为它不会在树的构建过程中就过早地丢弃信息。
  3. **避免过度约束：** 预剪枝可能在树的早期阶段就限制了模型的生长，导致了欠拟合。后剪枝则允许树在一定程度上过拟合，然后通过剪枝来平衡模型的复杂度，避免了过度的约束。
  4. **动态调整：** 后剪枝使得模型的剪枝决策可以根据验证集的性能动态调整，而不是在树的构建过程中就静态地决定。这样可以更好地适应不同数据集和问题的特性。